In [17]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 
import random
import os.path

class UpworkDataFormatter:
    
    # Select random number from user count 
    # Check that we were able to download the photo, the data exists, gender is not "unknown"  
    # Format the link to the photo 
    # Write to CSV 
    
    def __init__(self):
        # Settings
        self.est_data_file_name = './csv_files/2estimated_gender_detector_2017_12_12_upwork_analysis_unitedstates_allskills.csv'
        self.ground_truth_data_file_name = './csv_files/2groundtruth_2017_12_12_upwork_analysis_unitedstates_allskills.csv'
        self.original_data_file_name = './csv_files/formatted_analysis_df_2017_12_12_upwork_analysis_unitedstates_allskills.csv' # Filename for all data
        self.resized_photo_path = './resized_profile_photos/resized_images_unitedstates_allskills_2017_12_12/'
        self.github_photo_path = 'https://raw.githubusercontent.com/efoongch/upwork-pay-by-gender/master/resized_profile_photos/resized_images_unitedstates_allskills_2017_12_12/'
        
        self.total_dataset_size = 55518 # insert dataset size here
        self.interrater_dataset_size = 200 # insert size of dataset for interrater reliablity
        self.random_users_list = []
        self.random_worker_id_list = []
        self.users_w_complete_data = []
        self.gender_list = []
        self.age_list = []
        self.unknown_count = 0
    
    def make_new_csv(self, column_name):
        est_data = './csv_files/2_6_2018_UpworkGroundTruth_1.csv'
        new_csv = './csv_files/estimated_final_gender_groundtruth1_2017_12_12_upwork_analysis_unitedstates_allskills.csv'
        
        df_est = pd.read_csv(est_data)
        df_original = pd.read_csv('./csv_files/formatted_analysis_df_2017_12_12_upwork_analysis_unitedstates_allskills.csv')
        
        count_list = []
        value_list = []
        age_list = []
        
        for index, row in df_est.iterrows():
            count = row['user_count']
            count_list.append(count)
        
        count_list = sorted(count_list) # Orders the list 
        count_list = count_list[:200]
        
        for index, row in df_original.iterrows():
            if row['user_count'] in count_list:
                value = row[column_name]
                age = row['new_age_range_id']
                print "This is user count: {0} and the est gender: {1}".format(row['user_count'], value)
                value_list.append(value)
                age_list.append(age)

        with open(new_csv, 'w') as csvfile:
            print "We have started a new csv file!"
            fieldnames = ['user_count', 'est_gender', 'est_age']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            count = 0
            for worker in count_list:
                user_count = count_list[count]
                est_gender = value_list[count]
                est_age = age_list[count]
                
                print "CSV: This is user count: {0} and est_gender: {1}".format(user_count, est_gender)
                
                writer.writerow({'user_count': user_count, 'est_gender': est_gender, 'est_age': est_age})
                count += 1
            print "Done with creating new CSV file"
            
    def save_to_csv(self):
        df = pd.read_csv(self.original_data_file_name)
        
        for index, row in df.iterrows():
            gender = row['gender_detector']
            if gender == "unknown":
                self.unknown_count += 1
            
        print "This is the count of the unknown gender profiles: {0}".format(self.unknown_count)
        
        # Create an array of random users
        while len(self.random_users_list) < self.interrater_dataset_size: 
            random_number = random.randint(1, self.total_dataset_size)
            age_range = df.iloc[random_number-1]['new_age_range_id']
            gender = df.iloc[random_number-1]['final_gender']
            worker_id = df.iloc[random_number-1]['worker_id_x']
    
            # If the file exists and there is no existing number in the dataset, then add it to the array 
            filename = self.resized_photo_path + str(random_number) + ".jpg"
            if (os.path.isfile(filename)): 
                if not random_number in self.random_users_list:
                    if (worker_id != "error"):
                        if (gender != "unknown"): # Gender and age were estimated
                            if (age_range != "error"):
                                if (age_range != "0"): # Because we will remove users who are obviously too young
                                    self.random_users_list.append(random_number)
                                    self.gender_list.append(gender)
                                    print "This is age range {0}".format(age_range)
                                    self.age_list.append(age_range)
        
        print "This is the list of random users we obtained: {0}".format(self.random_users_list)
        print "This is the age list: {0}".format(self.age_list)
        print "This is the number of unknown gender profiles: {0}".format(self.unknown_count)
        
        # Find the relevant worker ids
        for worker in self.random_users_list:
            worker_id = df.iloc[worker-1]['worker_id_x']
            print worker_id
            self.random_worker_id_list.append(worker_id)
        
        print "This is the list of random worker ids: {0}".format(self.random_worker_id_list)
        
        # Start writing CSV file of estimated values
        with open(self.est_data_file_name, 'w') as csvfile:
            print "We have started a new csv file!"
            fieldnames = ['user_count', 'worker_id', 'photo_link', 'est_gender', 
                          'est_age_range']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            count = 1
            for worker in self.random_worker_id_list:
                print "Now writing data for random user: {0}".format(count)
                user_count = self.random_users_list[count-1]
                worker_id = self.random_worker_id_list[count-1]
                photo_path = self.github_photo_path + str(self.random_users_list[count-1]) + ".jpg"
                formatted_photo_path = '=IMAGE("' + photo_path + '", 1)'# Formatted for Google Sheets
                est_gender = self.gender_list[count-1]
                est_age_range = self.age_list[count-1]
                
                writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'photo_link': formatted_photo_path, 'est_gender': est_gender, 
                                 'est_age_range': est_age_range })
                count += 1
        
        # Make CSV file for ground truth 
        with open(self.ground_truth_data_file_name, 'w') as csvfile:
            print "We have started a new csv file!"
            fieldnames = ['user_count', 'worker_id', 'photo_link', 'truth_gender', 'truth_age_range']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            count = 1
            for worker in self.random_worker_id_list:
                print "Now writing data for random user: {0}".format(count)
                user_count = self.random_users_list[count-1]
                worker_id = self.random_worker_id_list[count-1]
                photo_path = self.github_photo_path + str(self.random_users_list[count-1]) + ".jpg"
                formatted_photo_path = '=IMAGE("' + photo_path + '", 1)'# Formatted for Google Sheets
                
                writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'photo_link': formatted_photo_path,
                                 'truth_gender': "", 'truth_age_range': ""})
                count += 1

    
myObject = UpworkDataFormatter()
myObject.make_new_csv('final_gender')

This is user count: 682 and the est gender: female
This is user count: 1883 and the est gender: female
This is user count: 1901 and the est gender: male
This is user count: 2230 and the est gender: male
This is user count: 2313 and the est gender: male
This is user count: 2524 and the est gender: male
This is user count: 2650 and the est gender: male
This is user count: 2679 and the est gender: male
This is user count: 3707 and the est gender: female
This is user count: 3745 and the est gender: female
This is user count: 3838 and the est gender: female
This is user count: 4617 and the est gender: male
This is user count: 4940 and the est gender: female
This is user count: 5041 and the est gender: female
This is user count: 5067 and the est gender: male
This is user count: 5229 and the est gender: male
This is user count: 5608 and the est gender: male
This is user count: 5677 and the est gender: male
This is user count: 5730 and the est gender: male
This is user count: 5866 and the est 